In [72]:
from langchain_text_splitters import CharacterTextSplitter
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator= " ",
    chunk_size=2000,  # Slightly larger chunk size
    chunk_overlap=200,  # Significant overlap to maintain context
)

In [73]:
split_docs = text_splitter.split_text(test_text)

In [74]:
len(split_docs)

29

In [19]:
CharacterTextSplitter??

Init signature:
CharacterTextSplitter(
    separator: 'str' = '\n\n',
    is_separator_regex: 'bool' = False,
    **kwargs: 'Any',
) -> 'None'
Source:        
class CharacterTextSplitter(TextSplitter):
    """Splitting text that looks at characters."""

    def __init__(
        self, separator: str = "\n\n", is_separator_regex: bool = False, **kwargs: Any
    ) -> None:
        """Create a new TextSplitter."""
        super().__init__(**kwargs)
        self._separator = separator
        self._is_separator_regex = is_separator_regex

    def split_text(self, text: str) -> List[str]:
        """Split incoming text and return chunks."""
        # First we naively split the large input into a bunch of smaller ones.
        separator = (
            self._separator if self._is_separator_regex else re.escape(self._separator)
        )
        splits = _split_text_with_regex(text, separator, self._keep_separator)
        _separator = "" if self._keep_separator else self._separator
        r

In [16]:
from html.parser import HTMLParser
class HyperlinkParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.hyperlinks = []

    def handle_starttag(self, tag, attrs):
        attrs = dict(attrs)

        if tag == "a" and "href" in attrs:   
            self.hyperlinks.append(attrs["href"])

In [17]:
from urllib.parse import urlparse
import urllib.request

def get_hyperlinks(url):

    # Try to open the URL and read the HTML
    try:
        # Open the URL and read the HTML
        with urllib.request.urlopen(url) as response:

            # If the response is not HTML, return an empty list
            if not response.info().get('Content-Type').startswith("text/html"):
                return []

            # Decode the HTML
            html = response.read().decode('utf-8')
    except Exception as e:
        print(e)
        return []

    # Create the HTML Parser and then Parse the HTML to get hyperlinks
    parser = HyperlinkParser()
    parser.feed(html)

    return parser.hyperlinks

In [39]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from selenium.webdriver.remote.webelement import WebElement
import time


In [40]:
#setting up selenium driver
service = Service()
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
driver = webdriver.Chrome(service=service, options=options)

In [58]:
HTTP_URL_PATTERN = r'^http[s]*://.+'
import re
from urllib.parse import urlparse, urljoin
from typing import Set

def is_valid_webpage_url(url: str) -> bool:
    """
    Check if the URL is a valid webpage link (not mailto, tel, javascript, anchor, etc.)
    
    Args:
        url (str): URL to check
        
    Returns:
        bool: True if URL is a valid webpage link, False otherwise
    """
    if not url:
        return False
        
    # Convert to lowercase for checking
    url_lower = url.lower().strip()
    
    # List of invalid URL patterns
    invalid_patterns = [
        'mailto:',
        'tel:',
        'javascript:',
        'data:',
        'ftp:',
        'file:',
        'whatsapp:',
        'sms:',
        'market:'
    ]
    
    # Check for invalid patterns
    if any(pattern in url_lower for pattern in invalid_patterns):
        return False
        
    # Check if it's just an anchor link
    if url_lower.startswith('#'):
        return False
        
    try:
        # Parse the URL
        parsed = urlparse(url)
        
        # Check if it has a valid scheme and netloc (domain)
        if not all([parsed.scheme in ['http', 'https'], parsed.netloc]):
            return False
            
        # Exclude URLs that end with file extensions we don't want
        invalid_extensions = [
            '.pdf', '.jpg', '.jpeg', '.png', '.gif', '.doc', '.docx',
            '.xls', '.xlsx', '.zip', '.tar', '.gz', '.exe', '.dmg'
        ]
        if any(url_lower.endswith(ext) for ext in invalid_extensions):
            return False
            
        return True
        
    except Exception:
        return False

def retry_on_stale_element(max_retries: int = 3, delay: float = 1):
    """
    Decorator to retry operations when StaleElementReferenceException occurs
    """
    def decorator(func):
        def wrapper(*args, **kwargs):
            for attempt in range(max_retries):
                try:
                    return func(*args, **kwargs)
                except StaleElementReferenceException:
                    if attempt == max_retries - 1:
                        raise
                    time.sleep(delay)
            return None
        return wrapper
    return decorator

@retry_on_stale_element()
def get_href_safely(element: WebElement) -> str:
    """
    Safely extract href attribute from an element with retry logic
    """
    return element.get_attribute("href")

def get_domain_hyperlinks(url: str, wait_time: int = 10) -> Set[str]:
    """
    Scrapes all unique webpage links from a webpage, filtering out non-webpage URLs.
    
    Args:
        url (str): The URL of the webpage to scrape
        wait_time (int): Maximum time to wait for page to load in seconds
        
    Returns:
        set: A set of unique absolute webpage URLs found on the page
    """
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)
    unique_links = set()
    
    try:
        # Load the page
        driver.get(url)
        
        # Wait for the page to load and stabilize
        try:
            WebDriverWait(driver, wait_time).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )
            # Additional wait for dynamic content
            time.sleep(2)
        except TimeoutException:
            print(f"Warning: Page took longer than {wait_time} seconds to load")
        
        # Execute JavaScript to get all href attributes
        links = driver.execute_script("""
            const links = document.getElementsByTagName('a');
            return Array.from(links).map(link => link.href).filter(href => href);
        """)
        
        # Process links
        base_domain = urlparse(url).netloc
        for href in links:
            try:
                if href and is_valid_webpage_url(href):
                    # Convert relative URLs to absolute URLs
                    absolute_url = urljoin(url, href)
                    unique_links.add(absolute_url)
            except Exception as e:
                print(f"Error processing link {href}: {str(e)}")
        
        # Backup method: try to get any links that might have been missed
        try:
            elements = WebDriverWait(driver, 3).until(
                EC.presence_of_all_elements_located((By.TAG_NAME, "a"))
            )
            
            for element in elements:
                try:
                    href = get_href_safely(element)
                    if href and is_valid_webpage_url(href):
                        absolute_url = urljoin(url, href)
                        unique_links.add(absolute_url)
                except Exception as e:
                    continue
                    
        except Exception as e:
            print(f"Backup link extraction encountered an error: {str(e)}")
            
        return unique_links
    
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return set()
    
    finally:
        driver.quit()

In [65]:
domains = get_domain_hyperlinks('https://lobste.rs/')

In [66]:

start = [item for item in domains if item.startswith('http')]


In [69]:
len(domains)

219

In [67]:
difference = list(set(domains) - set(start))
print(difference)  # Output: [1, 2, 5]


[]


In [ ]:
url = 'https://techcommunity.microsoft.com/blog/aiplatformblog/introducing-phi-4-microsoft%E2%80%99s-newest-small-language-model-specializing-in-comple/4357090'

In [34]:
from bs4 import BeautifulSoup
import requests


In [35]:
def extract_domain_name(url):
    # Parse the URL
    parsed_url = urlparse(url)
    # Get the domain (hostname)
    hostname = parsed_url.hostname or parsed_url.path
    # Remove subdomains and top-level domains (TLDs)
    domain_parts = hostname.split('.')
    if len(domain_parts) > 2:
        # For domains with subdomains like news.ycombinator.com
        return domain_parts[-2]
    elif len(domain_parts) == 2:
        # For domains without subdomains like lobste.rs
        return domain_parts[0]
    else:
        # Return as is (rare case)
        return hostname

In [36]:
def remove_elements_with_few_words(arr):
    # Filter out elements where the number of words is less than 3
    return [element for element in arr if len(element.split()) >= 3]

In [37]:
def clean_body_content(body_content):
    print("Type of body content: ", type(body_content))
    soup = BeautifulSoup(body_content, "html.parser")

    for script_or_style in soup(["script", "style"]):
        script_or_style.extract()

    # Get text or further process the content
    cleaned_content = soup.get_text(separator="\n")
    # print(cleaned_content)
    cleaned_content = "\n".join(
        line.strip() for line in cleaned_content.splitlines() if line.strip()
    )

    cleaned_content_array = cleaned_content.split('\n')
    filtered_cleaned_content_array = remove_elements_with_few_words(cleaned_content_array)
    filtered_cleaned_content = " ".join(filtered_cleaned_content_array)

    return filtered_cleaned_content

In [38]:
def get_webtext(url):
    local_domain = extract_domain_name(url)
    print("Local domain: ", local_domain)

    #Hyperlinks in the URL
    url_hyperlink = get_domain_hyperlinks(local_domain=local_domain, url=url)
    print("URL Hyperlinks: ", url_hyperlink)

    # Get the text from the URL using BeautifulSoup
    soup = BeautifulSoup(requests.get(url).text, "html.parser")


    text = str(soup.body())
    cleaned_body_content = clean_body_content(text)
    #print("Cleaned body content: ", cleaned_body_content)
        
    #take_screenshot(url, local_domain)    
    return cleaned_body_content

In [40]:
text = get_webtext('https://techcommunity.microsoft.com/blog/aiplatformblog/introducing-phi-4-microsoft%E2%80%99s-newest-small-language-model-specializing-in-comple/4357090')

Local domain:  microsoft
URL Hyperlinks:  ['https://microsoft/blog/aiplatformblog/introducing-phi-4-microsoft%E2%80%99s-newest-small-language-model-specializing-in-comple/4357090/replies/4357488', 'https://huggingface.co/collections/microsoft/phi-3-6626e15e9585a200d2d761e3', 'https://support.microsoft.com/contactus', 'https://microsoft/blog/aiplatformblog/introducing-phi-4-microsoft%E2%80%99s-newest-small-language-model-specializing-in-comple/4357090/replies/4357361', 'https://blogs.windows.com/windowsexperience/2024/12/06/phi-silica-small-but-mighty-on-device-slm', 'https://microsoft/t5/s/gxcuf89792/auth/oidcss/sso_login_redirect/provider/default?referer=https%3A%2F%2Ftechcommunity.microsoft.com%2Fblog%2Faiplatformblog%2Fintroducing-phi-4-microsoft%25E2%2580%2599s-newest-small-language-model-specializing-in-comple%2F4357090', 'https://www.microsoft.com/en-us/microsoft-teams/group-chat-software', 'https://account.microsoft.com', 'https://www.microsoft.com/en-us/microsoft-cloud', 'https

In [41]:
text

"Skip to content Microsoft Community Hub Artificial Intelligence and Machine Learning AI - AI Platform Blog AI - AI Platform Blog 2 MIN READ Introducing Phi-4: Microsoft’s Newest Small Language Model Specializing in Complex Reasoning Dec 13, 2024 Learn about Phi-4, the latest small language model in Phi family, that offers high quality results at a small size (14B parameters). Today we are introducing , our 14B parameter state-of-the-art small language model (SLM) that excels at complex reasoning in areas such as math, in addition to conventional language processing. Phi-4 is the latest member of our Phi family of small language models and demonstrates what’s possible as we continue to probe the boundaries of SLMs. Phi-4 is currently available on Azure AI Foundry under a Microsoft Research License Agreement (MSRLA) and will be available on Phi-4 outperforms comparable and larger models on math related reasoning due to advancements throughout the processes, including the use of high-qua

In [1]:
!pip install goose3

  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=0386f88f8a2a03633df1bad5b6107bdf222b4cec86d991e470101f3faa0734ab
  Stored in directory: /home/antash/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [ ]:
url = 'https://techcommunity.microsoft.com/blog/aiplatformblog/introducing-phi-4-microsoft%E2%80%99s-newest-small-language-model-specializing-in-comple/4357090'

In [2]:
from goose3 import Goose

g = Goose()

In [78]:
article = g.extract(url = 'https://ghostarchive.org/search?term=https%3A%2F%2Fblogtitle.github.io%2Fgo-advanced-concurrency-patterns-part-4-unlimited-buffer-channels%2F')

In [79]:
article.title

'Archives for https://blogtitle.github.io/go-advanced-concurrency-patterns-part-4-unlimited-buffer-channels/ - Ghostarchive'

In [80]:
article.cleaned_text

'Switch to the old website look'

In [81]:
article.infos

{'meta': {'description': '',
  'lang': 'en',
  'keywords': '',
  'favicon': '/favicon.ico',
  'canonical': 'https://ghostarchive.org/search?term=https%3A%2F%2Fblogtitle.github.io%2Fgo-advanced-concurrency-patterns-part-4-unlimited-buffer-channels%2F',
  'encoding': 'utf-8'},
 'image': None,
 'domain': 'ghostarchive.org',
 'title': 'Archives for https://blogtitle.github.io/go-advanced-concurrency-patterns-part-4-unlimited-buffer-channels/ - Ghostarchive',
 'cleaned_text': 'Switch to the old website look',
 'opengraph': {},
 'tags': [],
 'tweets': [],
 'movies': [],
 'links': [],
 'authors': [],
 'publish_date': None}

In [32]:
top_image = article.schema

In [34]:
top_image

In [35]:
article.infos

{'meta': {'description': "A paper from Anthropic's Alignment Science team on Alignment Faking in AI large language models",
  'lang': 'en',
  'keywords': '',
  'favicon': '/favicon.ico',
  'canonical': 'https://www.anthropic.com/research/alignment-faking/',
  'encoding': 'utf-8'},
 'image': None,
 'domain': 'www.anthropic.com',
 'title': 'Alignment faking in large language models',
 'cleaned_text': "Most of us have encountered situations where someone appears to share our views or values, but is in fact only pretending to do so—a behavior that we might call “alignment faking”. Alignment faking occurs in literature: Consider the character of Iago in Shakespeare’s Othello, who acts as if he’s the eponymous character’s loyal friend while subverting and undermining him. It occurs in real life: Consider a politician who claims to support a particular cause in order to get elected, only to drop it as soon as they’re in office.\n\nCould AI models also display alignment faking? When models are

In [1]:
text = "A collision between a passenger ferry and a navy speedboat near Mumbai, India, has resulted in at least 13 deaths, with 101 people rescued, as an investigation is underway into the incident that occurred while the ferry was en route to the Elephanta Caves tourist destination."

In [2]:
from groq import Groq
import os

client_groq = Groq(
  api_key=os.environ.get("GROQ_API_KEY")
)

def check_url_type(article_text):
    system_prompt = """
      Classify the given summary as either 'IsArticle' or 'NotArticle'. Base your classification on the presence of a clear topic, relevant details, organized structure, formal tone, and neutral perspective. Respond with one of the following, and only this exact classification:
        - IsArticle
        - NotArticle
      Wait for the user to provide the summary text.
    """
    message = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": article_text},
    ]
    completion = client_groq.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=message,
        temperature=1,
        top_p=1,
        max_tokens=1024,
        stream=False
    )
    result = completion.choices[0].message.content
    print("Response LLM: ", result)

    return result

In [3]:
check_url_type(text)

Response LLM:  IsArticle


'IsArticle'

In [6]:
from urllib.parse import urlparse
import urllib.request

def extract_domain_name(url):
    # Parse the URL
    parsed_url = urlparse(url)
    # Get the domain (hostname)
    hostname = parsed_url.hostname or parsed_url.path
    # Remove subdomains and top-level domains (TLDs)
    domain_parts = hostname.split('.')
    if len(domain_parts) > 2:
        # For domains with subdomains like news.ycombinator.com
        return domain_parts[-2]
    elif len(domain_parts) == 2:
        # For domains without subdomains like lobste.rs
        return domain_parts[0]
    else:
        # Return as is (rare case)
        return hostname

In [7]:
extract_domain_name('https://lobste.rs/')

'lobste'

In [13]:
def get_domain_hyperlinks(local_domain, url):
    clean_links = []
    for link in set(get_hyperlinks(url)):
        clean_link = None

        # If the link is a URL, check if it is within the same domain
        if re.search(HTTP_URL_PATTERN, link):
            # Parse the URL and check if the domain is the same
            url_obj = urlparse(link)
            # if url_obj.netloc == local_domain:
            #     clean_link = link
            clean_link = link

        # If the link is not a URL, check if it is a relative link
        else:
            if link.startswith("/"):
                link = link[1:]
            elif link.startswith("#") or link.startswith("mailto:"):
                continue
            clean_link = "https://" + local_domain + "/" + link

        if clean_link is not None:
            if clean_link.endswith("/"):
                clean_link = clean_link[:-1]
            clean_links.append(clean_link)

    # Return the list of hyperlinks that are within the same domain
    return clean_links


In [ ]:
def get_summary(url, depth=1, max_depth=2, processed_urls=None):
    """
    Recursively scrape a URL, get content summaries, and navigate nested links up to a given depth.
    
    :param url: The starting URL to scrape.
    :param depth: Current depth of recursion.
    :param max_depth: Maximum recursion depth allowed.
    :param processed_urls: Set of URLs already processed to avoid redundancy.
    :return: A list of summaries for the current and nested URLs.
    """
    if depth > max_depth:
        # Stop recursion when max depth is reached
        return []

    if processed_urls is None:
        # Initialize the set to track processed URLs
        processed_urls = set()

    if url in processed_urls:
        # Skip already processed URLs
        print(f"Skipping already processed URL: {url}")
        return []

    print(f"Processing URL at depth {depth}: {url}")
    processed_urls.add(url)

    local_domain = extract_domain_name(url)
    print("Local-domain name:", local_domain)

    try:
        # Fetch and clean the content from the main URL
        cleaned_body_content = get_webtext(url)

        # Get the summary and page type
        #llm_response, url_type = call_llm(cleaned_body_content)

        # Initialize the parent response
        parent_response = {"url": url, "page_type": url_type, "url_summary": llm_response}
        all_llm_response = [parent_response]

        # If the page type is valid, retrieve hyperlinks for nested scraping
        url
        if url_type:
            url_hyperlinks = get_domain_hyperlinks(local_domain=local_domain, url=url)
            print(f"Total Number of hyperlinks found: {len(url_hyperlinks)}")
            unique_links = set(url_hyperlinks) - processed_urls  # Filter only unprocessed links

            for link in unique_links:
                try:
                    print(f"Processing nested link: {link}")

                    # Recursively fetch summaries for nested links
                    nested_summary = get_summary(link, depth=depth + 1, max_depth=max_depth, processed_urls=processed_urls)
                    if nested_summary:
                        all_llm_response.extend(nested_summary)

                except Exception as e:
                    print(f"Error processing link {link}: {e}")
                    continue  # Skip to the next link in case of an error

        return all_llm_response

    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        return []

In [26]:
get_summary('https://edition.cnn.com/2024/12/18/asia/elephanta-caves-navy-boat-collision-mumbai-intl-latam/index.html')

Local-domain name:  cnn
Error processing URL https://edition.cnn.com/2024/12/18/asia/elephanta-caves-navy-boat-collision-mumbai-intl-latam/index.html: name 'get_webtext' is not defined


[]

In [9]:
import sqlite3

def get_history():

    try:
        conn = sqlite3.connect('scraper.db')
        cursor = conn.cursor()

        cursor.execute('SELECT distinct url FROM articles where isarticle = \'IsArticle\' ')
        rows = cursor.fetchall()

        # Structure the data as a list of dictionaries
        history = []
        for row in rows:
            entry = row[0]
            history.append(entry)

        # Close the database connection
        conn.close()

        return history
    
    except Exception as e:
        return None


In [10]:
history = get_history()

In [11]:
history

['https://wiki.alopex.li/SurveyOfSystemLanguages2024',
 'https://web.archive.org/web/3/https%3A%2F%2Fwww.scylladb.com%2F2024%2F12%2F18%2Fwhy-were-moving-to-a-source-available-license%2F',
 'https://web.archive.org/web/3/https%3A%2F%2Fgithub.com%2Fergochat%2Fergo',
 'https://web.archive.org/web/3/https%3A%2F%2Fgvolpe.com%2Fblog%2Fhome-manager-dotfiles-management%2F',
 'https://github.com/garyexplains/piccolo_os_v1',
 'https://chapel-lang.org/blog/posts/announcing-chapel-2.3',
 'https://web.archive.org/web/3/https%3A%2F%2Fwww.netbsd.org%2Freleases%2Fformal-10%2FNetBSD-10.1.html',
 'https://web.archive.org/web/3/https%3A%2F%2Fhorstmann.com%2Funblog%2F2024-12-11%2Findex.html',
 'https://www.netbsd.org/releases/formal-10/NetBSD-10.1.html',
 'https://seastar.io/shared-nothing',
 'https://slint.dev/blog/slint-1.9-released',
 'https://web.archive.org/web/3/https%3A%2F%2Fwiki.alopex.li%2FSurveyOfSystemLanguages2024',
 'https://blogtitle.github.io/go-advanced-concurrency-patterns-part-4-unlimite

In [12]:
import sqlite3
import logging

logger = logging.getLogger(__name__)

def scrape_and_store():

    try:        
        
        get_url = history        

        conn = sqlite3.connect('scraper.db')
        cursor = conn.cursor()

        update_query  = '''
                    UPDATE articles
                    SET title = ?
                    where url = ?; 
        '''
        
        logger.warn(
            f"Connection Done: {conn}"
        )

        try:
            for url in get_url:
                article = g.extract(url = url)    
                title = article.title               
                cursor.execute(update_query,(title, url))
            conn.commit()
        
        finally:
            conn.close()

    except Exception as e:
        logger.error(f"Error in scrape_and_store: {e}")
        raise e

In [14]:
scrape_and_store()

/tmp/ipykernel_19323/537330283.py:21: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  logger.warn(
Connection Done: <sqlite3.Connection object at 0x7f51ca8a3010>


In [4]:
!pip install huggingface-hub

In [3]:
from llama_cpp import Llama

In [47]:
from llama_cpp import Llama

llm = Llama.from_pretrained(
	repo_id="antash420/Llama-3.1-8B-Instruct-Q5_K_S-GGUF",
	filename="llama-3.1-8b-instruct-q5_k_s.gguf",
	n_gpu_layers=1,
    flash_attn=True
)


llama_model_loader: loaded meta data with 33 key-value pairs and 292 tensors from /home/antash/.cache/huggingface/hub/models--antash420--Llama-3.1-8B-Instruct-Q5_K_S-GGUF/snapshots/939711e32853e6c2f7b56b3ae96c57e04f8bf8ec/./llama-3.1-8b-instruct-q5_k_s.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.1 8B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Llama-3.1
llama_model_loader: - kv   5:                         general.size_label str              = 8B
llama_mod

In [48]:
def check_url_type(article_text):
    system_prompt = """
    Classify the given summary as either 'IsArticle' or 'NotArticle'. Base your classification on the presence of a clear topic, relevant details, organized structure, formal tone, and neutral perspective. Respond with one of the following, and only this exact classification:
    - IsArticle
    - NotArticle
    Wait for the user to provide the summary text.
    """

    response = llm.create_chat_completion(
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": article_text},
        ]
    )
    print(response)
    return response

In [52]:
text = "Piccolo OS is a basic, co-operative multitasking operating system designed for the Raspberry Pi Pico, primarily as a teaching tool to demonstrate co-operative multitasking and Arm Cortex-M0+ fundamentals. It features a simple round-robin scheduling mechanism and supports multiple tasks with separate stacks, relying on tasks to voluntarily yield control back to the kernel for task switching."

In [53]:
response = check_url_type(text)

Llama.generate: 104 prefix-match hit, remaining 77 prompt tokens to eval
llama_perf_context_print:        load time =    9527.79 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    77 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     2 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    5043.29 ms /    79 tokens


{'id': 'chatcmpl-6142a50e-2ea6-43c3-b2d4-b58b81720014', 'object': 'chat.completion', 'created': 1734800699, 'model': '/home/antash/.cache/huggingface/hub/models--antash420--Llama-3.1-8B-Instruct-Q5_K_S-GGUF/snapshots/939711e32853e6c2f7b56b3ae96c57e04f8bf8ec/./llama-3.1-8b-instruct-q5_k_s.gguf', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'IsArticle'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 181, 'completion_tokens': 2, 'total_tokens': 183}}


In [54]:
response['choices'][0]['message']['content']

'IsArticle'